# Window LAG

## COVID-19 Data
Notes on the data: This data was assembled based on work done by [Rodrigo Pombo](https://github.com/pomber/covid19) based on [John Hopkins University](https://systems.jhu.edu/research/public-health/ncov/), based on [World Health Organisation](https://www.who.int/health-topics/coronavirus). The data was assembled 21st April 2020 - there are no plans to keep this data set up to date.

In [1]:
import os
import pandas as pd
import findspark
os.environ['SPARK_HOME'] =  '/opt/spark'
findspark.init()

from pyspark.sql import SparkSession
ss = (SparkSession.builder.appName('app00')
      .config('spark.sql.warehouse.dir', 'hdfs://quickstart.cloudera:8020/user/hive/warehouse')
      .config('hive.metastore.uris', 'thrift://quickstart.cloudera:9083')
      .enableHiveSupport().getOrCreate())

 ····


## Window Function
The SQL Window functions include LAG, LEAD, RANK and NTILE. These functions operate over a "window" of rows - typically these are rows in the table that are in some sense adjacent.

In [2]:
covid = pd.read_sql_table('covid', engine)
world = pd.read_sql_table('world', engine)

## 1. Introducing the `covid` table

The example uses a WHERE clause to show the cases in 'Italy' in March.

**Modify the query to show data from Spain**

In [3]:
(covid.assign(day=covid['whn'].dt.day)
    .loc[(covid['name']=='Spain') & (covid['whn'].dt.month==3),
        ['name', 'day', 'confirmed', 'deaths', 'recovered']]
    .sort_values('day'))

,name,day,confirmed,deaths,recovered
13719,Spain,1,84,0,2
13720,Spain,2,120,0,2
13721,Spain,3,165,1,2
13722,Spain,4,222,2,2
13723,Spain,5,259,3,2
13724,Spain,6,400,5,2
13725,Spain,7,500,10,30
13726,Spain,8,673,17,30
13727,Spain,9,1073,28,32
13728,Spain,10,1695,35,32


## 2. Introducing the LAG function

The LAG function is used to show data from the preceding row or the table. When lining up rows the data is partitioned by country name and ordered by the data whn. That means that only data from Italy is considered.

**Modify the query to show confirmed for the day before.**

In [4]:
a = (covid.assign(day=covid['whn'].dt.day, mo=covid['whn'].dt.month)
    .loc[(covid['name']=='Italy') & (covid['whn'].dt.month==3),
        ['name', 'day', 'confirmed', 'mo']])
(a.assign(lag_cfrm=a['confirmed'].shift(1))
    .sort_values('day'))

,name,day,confirmed,mo,lag_cfrm
7329,Italy,1,1694,3,NaN
7330,Italy,2,2036,3,1694.0
7331,Italy,3,2502,3,2036.0
7332,Italy,4,3089,3,2502.0
7333,Italy,5,3858,3,3089.0
7334,Italy,6,4636,3,3858.0
7335,Italy,7,5883,3,4636.0
7336,Italy,8,7375,3,5883.0
7337,Italy,9,9172,3,7375.0
7338,Italy,10,10149,3,9172.0


### LAG operation

Here is the correct query showing the cases for the day before:

```sql
SELECT name, DAY(whn), confirmed,
   LAG(confirmed, 1) OVER (partition by name ORDER BY whn) AS lag
 FROM covid
WHERE name = 'Italy'
AND MONTH(whn) = 3
ORDER BY whn
```

Notice how the values in the LAG column match the value of the row diagonally above and to the left.

name | DAY(whn) | confirmed | dbf
------|---|------|----------
Italy | 1 | **1694** | null
Italy | 2 | 2036 | **1694**
Italy | 3 | 2502 | 2036
Italy | 4 | 3089 | 2502
Italy | 5 | **3858** | 3089
Italy | 6 | 4636 | **3858**
Italy | 7 | 5883 | 4636
Italy | 8 | 7375 | 5883
Italy | 9 | 9172 | 7375
Italy | 10 | 10149 | 9172
... | | |

## 3. Number of new cases

The number of confirmed case is cumulative - but we can use LAG to recover the number of new cases reported for each day.

**Show the number of new cases for each day, for Italy, for March.**

In [5]:
a = covid.assign(day=covid['whn'].dt.day)[
    (covid['name']=='Italy') & (covid['whn'].dt.month==3)]
a.assign(new=a['confirmed']-a['confirmed'].shift(1))[['name', 'day', 'new']]

,name,day,new
7329,Italy,1,NaN
7330,Italy,2,342.0
7331,Italy,3,466.0
7332,Italy,4,587.0
7333,Italy,5,769.0
7334,Italy,6,778.0
7335,Italy,7,1247.0
7336,Italy,8,1492.0
7337,Italy,9,1797.0
7338,Italy,10,977.0


## 4. Weekly changes

The data gathered are necessarily estimates and are inaccurate. However by taking a longer time span we can mitigate some of the effects.

You can filter the data to view only Monday's figures **WHERE WEEKDAY(whn) = 0**.

**Show the number of new cases in Italy for each week - show Monday only.**

In [6]:
a = (covid[(covid['name']=='Italy') & (covid['whn'].dt.weekday==6)]).copy()
(a.assign(new = a['confirmed'] - a['confirmed'].shift(1))
        .loc[:, ['name', 'whn', 'new']])

,name,whn,new
7294,Italy,2020-01-26,NaN
7301,Italy,2020-02-02,2.0
7308,Italy,2020-02-09,1.0
7315,Italy,2020-02-16,0.0
7322,Italy,2020-02-23,152.0
7329,Italy,2020-03-01,1539.0
7336,Italy,2020-03-08,5681.0
7343,Italy,2020-03-15,17372.0
7350,Italy,2020-03-22,34391.0
7357,Italy,2020-03-29,38551.0


## 5. LAG using a JOIN

You can JOIN a table using DATE arithmetic. This will give different results if data is missing.

**Show the number of new cases in Italy for each week - show Monday only.**

In the sample query we JOIN this week tw with last week lw using the DATE_ADD function.

In [7]:
# SELECT tw.name, TO_CHAR(tw.whn,'yyyy-mm-dd'), 
#   tw.confirmed - lw.confirmed AS new
#     FROM covid tw LEFT JOIN covid lw ON 
#     lw.whn + INTERVAL '1w' = tw.whn AND tw.name=lw.name
#     WHERE tw.name = 'Italy' AND EXTRACT(DOW FROM tw.whn)=0
#     ORDER BY tw.whn;
import datetime
a = (covid[(covid['name']=='Italy') & (covid['whn'].dt.weekday==6)]).copy()
b = (a.merge(a.assign(whn=covid['whn'] + datetime.timedelta(weeks=1)), 
        how='left', left_on=['whn', 'name'], right_on=['whn', 'name']))
b.assign(new=b['confirmed_x']-b['confirmed_y'])[['name', 'whn', 'new']]

,name,whn,new
0,Italy,2020-01-26,NaN
1,Italy,2020-02-02,2.0
2,Italy,2020-02-09,1.0
3,Italy,2020-02-16,0.0
4,Italy,2020-02-23,152.0
5,Italy,2020-03-01,1539.0
6,Italy,2020-03-08,5681.0
7,Italy,2020-03-15,17372.0
8,Italy,2020-03-22,34391.0
9,Italy,2020-03-29,38551.0


## 6. RANK()

The query shown shows the number of confirmed cases together with the world ranking for cases.

United States has the highest number, Spain is number 2...

Notice that while Spain has the second highest confirmed cases, Italy has the second highest number of deaths due to the virus.

**Include the ranking for the number of deaths in the table. Only include countries with a population of at least 10 million.**

In [8]:
a = covid.merge(world[['name', 'population']], left_on='name', right_on='name')
(a.assign(rc1=a['confirmed'].rank(ascending=False),
          rc2=a['deaths'].rank(ascending=False))
    .loc[(a['whn']=='2020-04-20') & (a['population']>=1e7),
        ['name', 'confirmed', 'rc1', 'deaths', 'rc2']]
    .sort_values('confirmed', ascending=False))

,name,confirmed,rc1,deaths,rc2
15479,United States,784326,1.0,42094,1.0
13769,Spain,200210,21.0,20852,17.0
7379,Italy,181228,27.0,24114,8.0
5399,France,156480,42.0,20292,21.0
5759,Germany,147065,52.0,4862,119.0
15389,United Kingdom,125856,73.0,16550,42.0
15029,Turkey,90980,107.0,2140,263.0
3239,China,83817,116.0,4636,126.5
7019,Iran,83505,120.0,5209,111.0
12059,Russia,47121,247.0,405,497.5


## 7. Infection rate

The query shown includes a JOIN t the world table so we can access the total population of each country and calculate infection rates (in cases per 100,000).

**Show the infect rate ranking for each country. Only include countries with a population of at least 10 million.**

In [9]:
# a was obtained in #6
b = (a[(a['whn']=='2020-04-20') & (a['population']>1e7)]).copy()
b['r_inf'] = round(1e5*b['confirmed']/b['population'])
(b.assign(rc=b['r_inf'].rank(method='min'))
    .sort_values('population', ascending=False)
    .loc[:, ['name', 'r_inf', 'rc']])

,name,r_inf,rc
3239,China,6.0,50.0
6839,India,1.0,23.0
15479,United States,238.0,87.0
6929,Indonesia,3.0,35.0
11159,Pakistan,4.0,41.0
2159,Brazil,19.0,65.0
10799,Nigeria,0.0,1.0
1259,Bangladesh,2.0,31.0
12059,Russia,32.0,71.0
9539,Mexico,7.0,54.0


## 8. Turning the corner

For each country that has had at last 1000 new cases in a single day, show the date of the peak number of new cases.

In [10]:
a = covid.assign(new=(covid['confirmed']-covid.groupby('name')['confirmed']
                      .shift(1)).fillna(0))
a['rc'] = a.groupby('name')['new'].rank(ascending=False, method='max')
(a.loc[(a['rc']==1) & (a['new']>1000), ['name', 'whn', 'new']]
    .sort_values(['whn', 'name']))

,name,whn,new
3172,China,2020-02-13,15136.0
7349,Italy,2020-03-21,6557.0
14191,Switzerland,2020-03-23,1321.0
7263,Israel,2020-03-25,1131.0
13743,Spain,2020-03-25,9630.0
874,Austria,2020-03-26,1321.0
5735,Germany,2020-03-27,6933.0
6998,Iran,2020-03-30,3186.0
2864,Canada,2020-04-05,2778.0
4489,Ecuador,2020-04-10,2196.0
